In [55]:
import os 
import sys 
from SourceFiles.teproteus import TEProteusAdmin as TepAdmin
import matplotlib.pyplot as plt
import numpy as np

In [56]:
# connect and initialize
admin = TepAdmin() #required to control PXI module
sid = 8 #PXI slot WDS found
inst = admin.open_instrument(slot_id=sid)
resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp) # Print *IDN
inst.send_scpi_cmd('*CLS; *RST')

Load TEProteus.dll from : C:\Windows\system32\TEProteus.dll
connected to: slot#: 8,


0

In [1]:
# initialize DAC
sampleRateDAC = 1E9
inst.send_scpi_cmd(':FREQ:RAST {0}'.format(sampleRateDAC))
inst.send_scpi_cmd(':SOUR:INT X2')

NameError: name 'inst' is not defined

In [58]:
# initialize EXT trigger (trigger for every channel need to be set independantly!)
# TRIG:LEV property of a trigger
# TRIG:STAT:SOUR:ENAB property of chan
# TRIG:STAT property of a selected trigger to an selected channel

inst.send_scpi_cmd(':INST:CHAN 1') 
inst.send_scpi_cmd(':TRIG:SOUR:ENAB TRG1')
inst.send_scpi_cmd(':TRIG:SEL TRG1')
inst.send_scpi_cmd(':TRIG:LEV 0.5')
inst.send_scpi_cmd(':TRIG:STAT ON')

inst.send_scpi_cmd(':INST:CHAN 2') 
inst.send_scpi_cmd(':TRIG:SOUR:ENAB TRG1')
inst.send_scpi_cmd(':TRIG:STAT ON')



0

In [59]:
#wavefore parameters

max_dac=65535 # Max Dac
half_dac=max_dac/2 # DC Lev
data_type = np.uint16 # DAC data type

segTime =np.int16(1E4)


 

 

dacWaveOn_I = np.ones(segTime)

dacWaveOn_I = dacWaveOn_I * max_dac

dacWaveOn_I = dacWaveOn_I.astype(data_type)

dacWaveOn_Q = np.zeros(segTime)

dacWaveOn_Q = dacWaveOn_Q * max_dac

dacWaveOn_Q = dacWaveOn_Q.astype(data_type)

# reshapes for IQIQIQIQIQ....


dacWaveOn_IQ = np.array([val for pair in zip(dacWaveOn_I, dacWaveOn_Q) for val in pair])


In [60]:
# channel 1 DUC
inst.send_scpi_cmd(':INST:CHAN {0}'.format(1)) 
inst.send_scpi_cmd(':MODE DUC')
inst.send_scpi_cmd(':IQM ONE')
inst.send_scpi_query(':SOUR:NCO:CFR1 100E6')

inst.send_scpi_cmd(':INST:CHAN {0}'.format(2)) 
inst.send_scpi_cmd(':MODE DUC')
inst.send_scpi_cmd(':IQM ONE')
inst.send_scpi_query(':SOUR:NCO:CFR1 100E6')

''

In [61]:
# select CH1
ch = 1 # everything after relates to CH 1 
segnum = 1
inst.send_scpi_cmd(':INST:CHAN {0}'.format(ch)) 
inst.send_scpi_cmd(':TRAC:DEL:ALL')
inst.send_scpi_cmd(':TRAC:DEF {0}, {1}'.format(segnum, len(dacWaveOn_IQ)))
inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))
inst.write_binary_data(':TRAC:DATA', dacWaveOn_IQ) # write, and wait while *OPC completes

inst.send_scpi_cmd(':TASK:COMP:LENG 1')
inst.send_scpi_cmd(':TASK:COMP:SEL 1')
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')

inst.send_scpi_cmd(':TASK:COMP:ENAB TRG1')
inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')
inst.send_scpi_cmd(':TASK:COMP:WRITE')
inst.send_scpi_cmd(':SOUR:FUNC:MODE TASK')
inst.send_scpi_cmd(':OUTP ON')

0

In [62]:
ch = 2 # everything after relates to CH 2
inst.send_scpi_cmd(':INST:CHAN {0}'.format(ch)) 
inst.send_scpi_cmd(':TASK:COMP:LENG 1')
inst.send_scpi_cmd(':TASK:COMP:SEL 1')
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')
inst.send_scpi_cmd(':TASK:COMP:ENAB TRG1')
inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')
inst.send_scpi_cmd(':TASK:COMP:WRITE')
inst.send_scpi_cmd(':SOUR:FUNC:MODE TASK')
inst.send_scpi_cmd(':OUTP ON')

0

In [63]:
# channel 2 DUC
inst.send_scpi_cmd(':MODE DUC')
inst.send_scpi_cmd(':IQM ONE')
inst.send_scpi_query(':SOUR:NCO:CFR1 100E6')

''

0